In [0]:
storageAccountName = "datalakeeb173f90c7e0bc50"
storageAccountAccessKey = ""
sasToken = "sv=2024-11-04&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2025-06-06T06:54:58Z&st=2025-06-05T22:54:58Z&spr=https&sig=rCr16f24u6jMldql1xRMuplwWvT646ItW%2FJurgEsQQ4%3D"


def mount_adls(blobContainerName):
    try:
      dbutils.fs.mount(
        source = "wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, storageAccountName),
        mount_point = f"/mnt/{storageAccountName}/{blobContainerName}",
        #extra_configs = {'fs.azure.account.key.' + storageAccountName + '.blob.core.windows.net': storageAccountAccessKey}
        extra_configs = {'fs.azure.sas.' + blobContainerName + '.' + storageAccountName + '.blob.core.windows.net': sasToken}
      )
      print("OK!")
    except Exception as e:
      print("Falha", e)

In [0]:
mount_adls('silver')

In [0]:
df_assistencias  = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/assistencias")
df_atores        = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/atores")
df_avaliacoes    = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/avaliacoes")
df_episodios     = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/episodios")
df_filmes        = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/filmes")
df_generos       = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/generos")
df_pagamentos    = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/pagamentos")
df_planos        = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/planos")
df_series        = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/series")
df_usuarios      = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/usuarios")

In [0]:
from pyspark.sql.functions import current_timestamp, lit

df_assistencias  = df_assistencias.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("assistencias"))
df_atores        = df_atores.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("atores"))
df_avaliacoes    = df_avaliacoes.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("avaliacoes"))
df_episodios     = df_episodios.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("episodios"))
df_filmes        = df_filmes.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("filmes"))
df_generos       = df_generos.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("generos"))
df_pagamentos    = df_pagamentos.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("pagamentos"))
df_planos        = df_planos.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("planos"))
df_series        = df_series.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("series"))
df_usuarios      = df_usuarios.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("usuarios"))


In [0]:
def renomear_colunas(diretorio):

    df = spark.read.format('delta').load(diretorio)

    tabela = diretorio.split('/')[-2]

    for coluna in df.columns:
        novo_nome = coluna.upper()
        novo_nome = novo_nome.replace("CD_", "CODIGO_")
        novo_nome = novo_nome.replace("VL_", "VALOR_")
        novo_nome = novo_nome.replace("DT_", "DATA_")
        novo_nome = novo_nome.replace("NM_", "NOME_")
        novo_nome = novo_nome.replace("DS_", "DESCRICAO_")
        novo_nome = novo_nome.replace("NR_", "NUMERO_")
        novo_nome = novo_nome.replace("_UF", "_UNIDADE_FEDERATIVA")
        df = df.withColumnRenamed(coluna, novo_nome)
        df = df.drop("DATA_HORA_BRONZE")
        df = df.drop("NOME_ARQUIVO")
        df = df.withColumn("NOME_ARQUIVO_BRONZE", lit(tabela))
        df = df.withColumn("DATA_ARQUIVO_SILVER", current_timestamp())

    df.write.format('delta').mode("overwrite").save(f"/mnt/{storageAccountName}/silver/{tabela}")

def renomear_arquivos_delta(diretorio):

    nomes_arquivos_delta = []

    arquivos = dbutils.fs.ls(diretorio)
    
    for arquivo in arquivos:
        nome_arquivo = arquivo.path
        renomear_colunas(nome_arquivo)

    return nomes_arquivos_delta

In [0]:
diretorio = f'/mnt/{storageAccountName}/bronze'

renomear_arquivos_delta(diretorio)